---
title: "Optuna"
subtitle: "is hot on Kaggle"
format: html
---

# Optuna - the choice of Kagglers

While doing [HF RL course](https://huggingface.co/deep-rl-course/) I bumped into Optuna.

Some vocab to get started:
 - `objective(trial)` function - function to optimize
 - trial - a single test, also an object passed to the objective function 
 - study - a set of trial at the end of which you get a suggestion of parameters to use
 - parameter - parameter to optimize
 - setting initial values for parameters to optimize:
   * optuna.trial.Trial.suggest_categeorical('name', ['list'])
   * optuna.trial.Trial.suggest_int('name', min, max)
   * optuna.trial.Trial.suggest_float('name', min, max)
   

Here is a quick summary of how to use it:

In [88]:
import optuna
optuna.logging.set_verbosity(optuna.logging.WARNING) # to supress unnecessary output

# with 100 trials find a minimum for a function (x-10)**2
def objective(trial):
    x = trial.suggest_float("x", -100, 100)
    return (x - 10)**2

study = optuna.create_study()
study.optimize(objective, n_trials=100)

study.best_params['x'] #pretty close

10.069140289320362

In [92]:
# however it won't do magic - here if you give it just 10 trial, 
# it will usually miss quite substantially

# however with 10 trials...
def objective(trial):
    x = trial.suggest_float("x", -100, 100)
    return (x - 10)**2

study = optuna.create_study()
study.optimize(objective, n_trials=10)

study.best_params['x'] # ... it is usually not so good

28.394424972253262

how optuna works internally is quite simple but ingenious: each call to `trial.suggest_*()` function already suggests a python object, so you can use it in your code straight away:

In [90]:
study = optuna.create_study()
def objective(trial):
    i = trial.suggest_int('x', 0, 100, step=10)
    print(f"next {i=}")
    return i

study.optimize(objective, n_trials=10)
study.best_params['x']

next i=0
next i=20
next i=40
next i=10
next i=90
next i=40
next i=80
next i=50
next i=90
next i=0


0

also it is quite interesting how the numbers are drawn from the space - this is all random and duplicates are possible. Especially if we have very limited space of available unique values as in here. This is not an implementation bug - here we deal with a single variable, but if we have multiple ones, it quite makes sense to try simillar values if we variate other parameters at the same time.

## Useful tricks!

#### Continue the optimization

In [91]:
# however with 10 trials...
def objective(trial):
    x = trial.suggest_float("x", -100, 100)
    return (x - 10)**2

study = optuna.create_study()
study.optimize(objective, n_trials=10)

study.best_params['x'] # ... it is usually not so good

# but training for another 10 iterations does the trick
study.optimize(objective, n_trials=10)

study.best_params['x'] # ok, now it is better :)

10.802422942208281

#### Use a db to tune on multiple machines

Or just run hiperparameter searches when your colab disconnects

Optuna allows for distrubuted trials

In [ ]:
#straight from optuna docs @ https://optuna.readthedocs.io/en/stable/tutorial/10_key_features/004_distributed.html
def objective(trial):
    x = trial.suggest_float("x", -10, 10)
    return (x - 2) ** 2


if __name__ == "__main__":
    study = optuna.load_study(
        study_name="distributed-example", storage="mysql://root@localhost/example"
    )
    study.optimize(objective, n_trials=100)

#### Searching strategies

TBD